# Imports 

In [ ]:
import numpy as np
import cmath
import time 
import sounddevice as sd
from scipy.io.wavfile import write

class dft():
  def __init__(self, x, fs, K=None):
    if (type(fs) != int) or (fs<=0):
      raise NameError('The frequency fs should be a positive integer.')
    if not isinstance(x, np. ndarray):
      raise NameError('The input signal x must be a numpy array.')
    if isinstance(x, np. ndarray):
      if x.ndim!=1:
        raise NameError('The input signal x must be a numpy vector array.')
    self.x=x
    self.fs=fs
    self.N=len(x)
    if K == None:
      K = len(self.x)

    if (type(K) != int) or (K <= 0) or (K < 0):
      raise NameError('K should be a positive integer.')
    self.K=K
    self.f=np.arange(self.K)*self.fs/self.K
    self.f_c=np.arange(-np.ceil(K/2)+1,np.floor(self.K/2)+1)*self.fs/self.K
  
  def changeK(self,K):
    if (type(K) != int) or (K <= 0) or (K <  0):
      raise NameError('K should be a positive integer.')
    old_K=self.K
    self.K=K
    self.f=np.arange(self.K)*self.fs/self.K
    self.f_c=np.arange(-np.ceil(K/2)+1,np.floor(self.K/2)+1)*self.fs/self.K
    print('The value of K was succefully change from %d to %d'%(old_K,self.K))
    pass

  def solve3(self):
      X=np.fft.fft(self.x,self.K)/np.sqrt(self.N);
      X_c=np.roll(X,np.int(np.ceil(self.K/2-1)))
      return [self.f,X,self.f_c,X_c]
  

class recordsound():
  def __init__(self, T, fs):
    self.T = T
    self.fs = fs
    
  def solve(self):
    print('start recording \n')
    voicerecording = sd.rec(int(self.T * self.fs), self.fs, 1)
    sd.wait()
    print('end recording \n')
    write('myvoice1.wav', self.fs, voicerecording.astype(np.float32))
    
    return voicerecording

class idft(object):
  def __init__(self, X, fs, N, K=None):
    self.X = X
    self.fs = fs
    self.N = N
    self.K = K
    if K == None:
      self.K = int(len(X)/2)-1

  def solve(self):
    x = np.zeros(self.N)
    for n in range(self.N):
      x[n] = 1/np.sqrt(self.N) * self.X[0] * np.exp(1j*2*cmath.pi*0*n/self.N)
      for k in range(1, self.K+1):
        x[n] = x[n] + 1/np.sqrt(self.N)*self.X[k]*np.exp(1j*2*cmath.pi*k*n/self.N)
        x[n] = x[n] + 1/np.sqrt(self.N)*np.conj(self.X[k])*np.exp(-1j*2*cmath.pi*k*n/self.N)
    Ts = 1/self.fs
    Treal = np.arange(self.N)*Ts
    return x, Treal

  def solve_ifft(self):
    x=np.fft.ifft(self.X,self.N)*np.sqrt(self.N)

    Ts= 1/self.fs
    Treal= np.arange(self.N)*Ts

    return x, Treal

In [ ]:
# Create training sets

def training_raw_digits(digit_recs, digits, T, fs, number_of_recordings):
  for digit in digits:
    individual_recordings = np.zeros((number_of_recordings, int(T*fs)))
    print('Say ' + str(digit) + ' when prompted')
    for i in range (number_of_recordings):
      time.sleep(1)
      record_digit = recordsound(T, fs)
      voice = record_digit.solve().reshape(int(T*fs))
      individual_recordings[i, :] = voice
    digit_recs.append(individual_recordings)

  np.save("recording_digits.npy", digit_recs)

def training_dft_digits(voice_recording_name, digits, fs):
  recordings = np.load(voice_recording_name)
  number_digits = len(recordings)
  number_recordings, N = recordings[0].shape
  DFTs = []
  DFTs_c = []
  for individual_digit in recordings:
    DFT_a = np.zeros((number_recordings, N))
    DFT_a_c = np.zeros((number_recordings, N))
    for i in range(number_recordings):
      selected_rec = individual_digit[i, :]
      normalization_i = np.linalg.norm(selected_rec)
      selected_rec = selected_rec / normalization_i
      DFT_selected_rec = dft(selected_rec, fs)
      [_, X, _, X_c] = DFT_selected_rec.solve3()
      DFT_a[i, :] = X
      DFT_a_c[i, :] = X_c
    DFTs.append(DFT_a)
    DFTs_c.append(DFT_a_c)
  np.save("spoken_digits_DFTs.npy", DFTs)
  np.save("spoken_digits_DFTs_c.npy", DFTs_c)


def test_digits(digit_recs, digits, T, fs, number_of_recordings):
    individual_recordings = np.zeros((number_of_recordings, int(T*fs)))
    print('Say a randomly chosen 1 or 2 when prompted')
    for i in range(number_of_recordings):
      time.sleep(1) 
      record_digit = recordsound(T, fs) 
      record_digit = recordsound(T, fs)
      voice = record_digit.solve().reshape(int(T*fs))
      individual_recordings[i, :] = voice
    digit_recs.append(individual_recordings)
    np.save("test_set.npy", individual_recordings) 

    test_set_audio = individual_recordings.reshape(T*fs*number_of_recordings)
    file_name = 'test_set_audio_rec.wav'
    write(file_name, fs, test_set_audio.astype(np.float32))



if __name__ == '__main__':
  T = 1
  fs = 8000
  N = 10
  digits = [1,2]
  digit_recs = []

  training_raw_digits(digit_recs, digits, T, fs, N)
  training_dft_digits("recording_digits.npy", digits, fs)
  test_digits(digit_recs, digits, T, fs, N) 


# Section 1

In [ ]:
if __name__ == '__main__':
  T = 1
  fs = 8000

  test_set = np.load("test_set_DFTs.npy")
  training_set_DFTs = np.abs(np.load("spoken_digits_DFTs.npy"))

  num_digits = len(training_set_DFTs)
  _, N = training_set_DFTs[0].shape
  average_spectra = np.zeros((num_digits, N), dtype=np.complex_)
  average_signal = np.zeros((num_digits, N), dtype=np.complex_)

  for i in range(num_digits):
    average_spectra[i, :] = np.mean(training_set_DFTs[i], axis=0)
    iDFT = idft(average_spectra[i, :], fs, N) 
    y_demodulation, Treal = iDFT.solve_ifft() 
    average_signal[i, :] = y_demodulation 

  num_recs, N = test_set.shape
  predicted_labels = np.zeros(num_recs)


  for i in range(num_recs):
    rec_i = test_set[i, :]
    energy_rec_i = np.linalg.norm(rec_i)
    rec_i /= energy_rec_i


    inner_prods = np.zeros(num_digits)
    
    for j in range(num_digits):
      inner_prods[j] = np.linalg.norm(np.convolve(rec_i, average_spectra[j, :]))**2 
    predicted_labels[i] = np.argmax(inner_prods) + 1

  print("Average spectrum comparison --- predicted labels: \n")
  print_matrix(predicted_labels[:, None], nr_decimals=0)

  np.save("test_set_DFTs.npy", DFTs_aux)
  np.save("predicted_labels_avg.npy", predicted_labels)


# Section 2

In [ ]:
if __name__ == '__main__':
  T = 1
  fs = 8000
  num_recs = 20
  
  training_set_DFTs = np.abs(np.load("spoken_digits_DFTs.npy"))

  num_digits = len(training_set_DFTs)
  _, N = training_set_DFTs[0].shape
  average_spectra = np.zeros((num_digits, N), dtype=np.complex_)
  average_signal = np.zeros((num_digits, N), dtype=np.complex_)

  for i in range(num_digits):
    average_spectra[i, :] = np.mean(np.absolute(training_set_DFTs[i]), axis=0)
    iDFT = idft(average_spectra[i, :], fs, N)
    y_demod, Treal = iDFT.solve_ifft()
    average_signal[i, :] = y_demod
  
  for i in range(num_recs):
    voice = sd.rec(int(T * fs), fs, 1)
    sd.wait()
    rec_i = voice.astype(np.float32)

    rec_i = rec_i[:,0]
    energy_rec_i = np.linalg.norm(rec_i)
    rec_i /= energy_rec_i
    
    inner_prods = np.zeros(num_digits)

    for j in range(num_digits):
      inner_prods[j] = np.linalg.norm(np.convolve(rec_i , average_signal[j, :], mode='valid'))**2

    print('The number is: ', np.argmax(inner_prods) + 1)
